### Neural Network

##### Import Cleaned Dataset

In [1]:
#Import cleaned dataset using load_sets function defined in src.data.sets
import sys
sys.path.insert(1, '..')
from src.data.sets import load_sets
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/beer_type')

##### Define Architecture

In [2]:
#Import `torch`, `torch.nn` as `nn` and `torch.nn.functional` as `F`# Solution:

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
#Instantiate `PytorchBinary` with the correct number of input feature and save it into a variable called `model`

# Solution:
import sys
sys.path.insert(1, '..')

from src.models.pytorch import PytorchMultiClass

model = PytorchMultiClass(X_train.shape[1])

In [4]:
#Import `get_device()` from `src.models.pytorch` and set `model` to use the device available

import sys
sys.path.insert(1,'..')

from src.models.pytorch import get_device

device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=12, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
)

In [5]:
#Print the architecture of `model`

print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=12, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
)


##### Train Model

In [6]:
#Instantiate a `nn.BCELoss()` and save it into a variable called `criterion` 

criterion = nn.CrossEntropyLoss()

In [7]:
#Instantiate a `torch.optim.Adam()` optimizer with the model's parameters and 0.001 as learning rate and save it into a variable called `optimizer`

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [8]:
#Instantiate a torch.optim.lr_scheduler.StepLR() scheduler that will decrease the learning rate by a coefficient of 0.9 for each epoch

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [9]:
#Create 2 variables called N_EPOCHS and BATCH_SIZE that will take respectively 5 and 32 as values

N_EPOCHS = 5
BATCH_SIZE = 32


In [10]:
#Import Dataset and DataLoader from torch.utils.data

from torch.utils.data import Dataset, DataLoader

#Import this class from `src/models/pytorch` and convert all sets to PytorchDataset
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)


In [11]:
#Create a for loop that will iterate through the specified number of epochs and will train the model with the training set and assess the performance on the validation set and print their scores

from src.models.pytorch import train_classification, test_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

KeyboardInterrupt: 

Our baseline model and NN perform similarly and poorly. This is likely due to the large number of classes that need to be predicted (104 for the target variable). 

In [12]:
#Save the model into the models folder

torch.save(model.state_dict(), "../models/neural_network.pt")

In [13]:
#Assess the model performance on the testing set and print its scores

test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.3f}')

	Loss: 0.1020	|	Accuracy: 0.232


In [14]:
import numpy as np
model=PytorchMultiClass(num_features=12)
model.load_state_dict(torch.load("../models/neural_network.pt"))
X_train_tensor=torch.Tensor(np.array(X_train))
y=model(X_train_tensor)